## Is it possible to predict  car rating based on its technical characteristics and user reviews?

### import data

In [ ]:
import requests as rq
from bs4 import BeautifulSoup
import pandas as pd
import re
# my data frame columns
data = pd.DataFrame(columns=['car_name','car_brand','year', 'price', 'rating_count', 'rating_link', 'Exterior_color', 'Interior_color', 'Drivetrain','mpg' ,'Fuel_type', 'Transmission', 'Engine', 'Engine_size','Mileage','li_tags_sum'])
# the for look will go to page by changing the index 'i', the rest is the same
for i in range(0,10):
    # Website in a variable
    website = 'https://www.cars.com/shopping/results/?page=' + str(i) + '&page_size=20&dealer_id=&keyword=&list_price_max=&list_price_min=&makes[]=&maximum_distance=all&mileage_max=&sort=best_match_desc&stock_type=used&year_max=&year_min=&zip='
    data_cars = rq.get(website)
    s = BeautifulSoup(data_cars.text, "html.parser")
    # the web site is a list of ads of cars, the ads contains some of the information
    # i choose to copy to a list[] all the href of each add and get the data from the full ad. 
    links = []
    for link in s.find_all("a",{'class':'vehicle-card-link js-gallery-click-link'}):
        links.append(link.get("href"))
        # main for for data after we got the list[page i]
    for link in links:
        linkf = "https://www.cars.com" + link
        data_car = rq.get(linkf)
        soup = BeautifulSoup(data_car.text, "html.parser")
        info = soup.find_all("dl", {'class':'fancy-description-list'})

        #car_name
        car_name = soup.find('h1').get_text()
        # the ad dont have the nake of the manufactures is a different label, i was forset to make a small function that will
        # get the name from the full discripshion;) good practic to know all the companies
        car_manufacturers = ["Acura", "Alfa Romeo", "Aston Martin", "Audi", "Bentley", "BMW", "Bugatti", "Buick", "Cadillac", "Chevrolet", "Chrysler", "Citroën", "Dodge", "Ferrari", "Fiat", "Ford", "Geely", "Genesis", "GMC", "Honda", "Hyundai", "Infiniti", "Jaguar", "Jeep", "Kia", "Koenigsegg", "Lamborghini", "Land Rover", "Lexus", "Lincoln", "Lotus", "Maserati", "Mazda", "McLaren", "Mercedes-Benz", "Mini", "Mitsubishi", "Nissan", "Pagani", "Polestar", "Porsche", "Ram", "Renault", "Rolls-Royce", "Smart", "Subaru", "Suzuki", "Tesla", "Toyota", "Volkswagen", "Volvo"]
        for manufacturer in car_manufacturers:
            if str(manufacturer).lower() in car_name.lower():
                car_brand = manufacturer
                break
        else:
            car_brand = 'Other'
            # olsa about the year of poduction
        year = car_name[:4]  # Extract the first four characters from the string
        year = int(year)  # Convert the extracted string to an integer

        #price
        price = soup.find('span',{'class': 'primary-price'}).get_text()

        try:
            rating_count = (soup.find('span',{'class': 'sds-rating__count'}).get_text())
        except:
            rating_count = 'n/a'
        try:
            rating_link = (soup.find('span',{'class': 'sds-rating__link sds-button-link'}).get_text())
        except:
            rating_link = 'n/a'
       # data : Exterior_color,Interior_color,Drivetrain,mpg,Fuel_type,Transmission,Engine,Mileage,sum of featurs
        comp_info = pd.DataFrame()
        cleaned_id_text = []
        for i in info[0].find_all('dt'):
            cleaned_id_text.append(i.text)
        cleaned_id__attrb_text = []
        for i in info[0].find_all('dd'):
            cleaned_id__attrb_text.append(i.text)
        comp_info['Id'] = cleaned_id_text
        comp_info['Attribute'] = cleaned_id__attrb_text
        Exterior_color = comp_info.iloc[0,1]
        Interior_color = comp_info.iloc[1,1]
        Drivetrain = comp_info.iloc[2,1]
        mpg_text = comp_info.iloc[3, 1]
               # the mpg data was a 2 numbers (23-42), i used the evg for my data.
        match = re.search(r'\n\n(\d+)[\u2013-](\d+)', mpg_text)
        if match:
  
            new1 = int(match.group(1))
            new2 = int(match.group(2))
            mpg = (new1 + new2) / 2 
        else:
            mpg=0
        Fuel_type = comp_info.iloc[4,1]
        Transmission = comp_info.iloc[5,1]
        Engine = comp_info.iloc[6,1]
        m = re.search(r'(\d+\.\d+)L', Engine)
        if m:
           Engine_size = m.group(1)
        Mileage = comp_info.loc[comp_info['Id'] == 'Mileage', 'Attribute'].iloc[0]
        try:
              table = soup.find('ul', {'class': 'sds-list sds-list--unordered all-features-list'})
              li_tags = table.find_all('li')
              li_tags_sum=(len(li_tags))
        except:
              li_tags_sum = 'n/a'

        data.loc[len(data)] = [car_name, car_brand,year,price, rating_count, rating_link, Exterior_color, Interior_color, Drivetrain, mpg,Fuel_type, Transmission, Engine,Engine_size ,Mileage,li_tags_sum]


### save_data

In [ ]:
from datetime import datetime
filename = 'test_F' + datetime.today().strftime('%Y-%m-%d')
data.to_csv(filename, index=False)

### import data 

In [ ]:
import pandas as pd
import requests as rq
from bs4 import BeautifulSoup
import pandas as pd
import re

# Read the CSV file into a pandas DataFrame
combined_df = pd.read_csv('finall_data.csv')




In [ ]:
df = combined_df.copy()
df = df.dropna(subset=['rating_count'])
df = df.dropna(subset=['rating_link'])

In [ ]:
df1 = combined_df.copy()

In [ ]:
df1

In [ ]:
df


### fix data

In [ ]:
# Convert "price" column to string
df['price'] = df['price'].astype(str)
# Convert "price" column to numeric, remove commas and dollar signs, and set non-numeric values to NaN
df['price'] = pd.to_numeric(df['price'].str.replace(',', '').str.replace('$', ''), errors='coerce')
# Replace "Not Priced" values with NaN
df.loc[df['price'] == 'Not Priced', 'price'] = 1
# Handle missing values appropriately (e.g., impute or drop)
df = df.dropna(subset=['price'])
# Filter rows where "price" is less than or equal to 1
df = df[df['price'] > 1]


In [ ]:
#Mileage
df['Mileage'] = df['Mileage'].astype(str).str.replace(',', '').str.replace('mi.', '')
df['Mileage'] = pd.to_numeric(df['Mileage'], errors='coerce').astype('Int64')

#### fix rating_link

In [ ]:
import re

def extract_number_from_string(string):
    """
    Extracts numbers from a string and returns as integer.
    """
    regex = r'\d+'
    matches = re.findall(regex, string)
    if matches:
        return int(matches[0])
    else:
        return None


In [ ]:
df['rating_link'] = df['rating_link'].astype(str).apply(lambda x: extract_number_from_string(x) if x else None)

### exploretory data analysis 

In [ ]:
#Exterior_color
df['Exterior_color'] = df['Exterior_color'].astype(str)
exterior_color_dict = {color: i for i, color in enumerate(df['Exterior_color'].unique())}
df['Exterior_color'] = df['Exterior_color'].replace(exterior_color_dict)

In [ ]:
#Interior_color
df['Interior_color'] = df['Interior_color'].astype(str)
Interior_color_dict = {color: i for i, color in enumerate(df['Interior_color'].unique())}
df['Interior_color'] = df['Interior_color'].replace(Interior_color_dict)

In [ ]:
#Drivetrain
df['Drivetrain'] = df['Drivetrain'].astype(str)
Drivetrain_dict = {Drivetrain: i for i, Drivetrain in enumerate(df['Drivetrain'].unique())}
df['Drivetrain'] = df['Drivetrain'].replace(Drivetrain_dict)

In [ ]:
#Fuel_type
df['Fuel_type'] = df['Fuel_type'].astype(str)
Fuel_type_dict = {Fuel_type: i for i, Fuel_type in enumerate(df['Fuel_type'].unique())}
df['Fuel_type'] = df['Fuel_type'].replace(Fuel_type_dict)

In [ ]:
car_manufacturers = ["Acura", "Alfa Romeo", "Aston Martin", "Audi", "Bentley", "BMW", "Bugatti", "Buick", "Cadillac", "Chevrolet", "Chrysler", "Citroën", "Dodge", "Ferrari", "Fiat", "Ford", "Geely", "Genesis", "GMC", "Honda", "Hyundai", "Infiniti", "Jaguar", "Jeep", "Kia", "Koenigsegg", "Lamborghini", "Land Rover", "Lexus", "Lincoln", "Lotus", "Maserati", "Mazda", "McLaren", "Mercedes-Benz", "Mini", "Mitsubishi", "Nissan", "Pagani", "Polestar", "Porsche", "Ram", "Renault", "Rolls-Royce", "Smart", "Subaru", "Suzuki", "Tesla", "Toyota", "Volkswagen", "Volvo", "Other"]     
# create a dictionary to map car brand names to numbers
car_brand_map = {brand: i for i, brand in enumerate(car_manufacturers)}
# replace the car brand names with their corresponding numbers
df['car_brand'] = df['car_brand'].replace(car_brand_map)

In [ ]:
#Transmission
df['Transmission'] = df['Transmission'].astype(str)
Transmission_dict = {Transmission: i for i, Transmission in enumerate(df['Transmission'].unique())}
df['Transmission'] = df['Transmission'].replace(Transmission_dict)

In [ ]:
bins = [0, 5000, 10000, 15000, 25000, 30000,35000,45000,70000,80000,90000,100000,150000,500000,2000000]
labels = [1,2,3,4,5,6,7,8,9,10,11,12,13,14]
df['price'] = pd.cut(df['price'], bins=bins, labels=labels, include_lowest=True)

In [ ]:
bins = [0, 10000, 15000, 20000, 25000, 30000,35000,50000,70000,80000,90000,100000,150000,500000,2000000]
labels = [1,2,3,4,5,6,7,8,9,10,11,12,13,14]
df['Mileage'] = pd.cut(df['Mileage'], bins=bins, labels=labels, include_lowest=True)

In [ ]:
bins = [0,1,2,3,4,5]
labels = [1,2,3,4,5]
df['rating_count'] = pd.cut(df['rating_count'], bins=bins, labels=labels, include_lowest=True)

In [ ]:
df

In [ ]:
df = df[df['rating_link'] > 10]

In [ ]:
df

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Choose a feature to plot as a pie chart
feature_to_plot = 'Fuel_type'

# Get the count of each category
value_counts = df1[feature_to_plot].value_counts()

# Filter out categories with a count lower than X
X = 500
value_counts = value_counts[value_counts >= X]

# Create the pie chart
plt.pie(value_counts, labels=value_counts.index, autopct='%1.1f%%')

# Add a title
plt.title('Fuel Type Distribution (count >= {})'.format(X))

# Show the plot
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Choose a feature to plot as a bar plot
feature_to_plot = 'car_brand'

# Get the count of each category
value_counts = df1[feature_to_plot].value_counts()

# Filter out categories with a count lower than X
X = 100
value_counts = value_counts[value_counts >= X]

# Create the bar plot
plt.barh(value_counts.index, value_counts)

# Add axis labels and a title
plt.xlabel('Count')
plt.ylabel(feature_to_plot)
plt.title('{} Distribution (count >= {})'.format(feature_to_plot.capitalize(), X))

# Show the plot
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Choose a feature to plot as a bar plot
feature_to_plot = 'rating_count'

# Get the count of each category
value_counts = df1[feature_to_plot].value_counts()

# Filter out categories with a count lower than X
X = 100
value_counts = value_counts[value_counts >= X]

# Create the bar plot
plt.barh(value_counts.index, value_counts)

# Add axis labels and a title
plt.xlabel('Count')
plt.ylabel(feature_to_plot)
plt.title('{} Distribution (count >= {})'.format(feature_to_plot.capitalize(), X))

# Show the plot
plt.show()


In [ ]:

import pandas as pd
import matplotlib.pyplot as plt

# Choose a feature to plot as a bar plot
feature_to_plot = 'Transmission'

# Get the count of each category
value_counts = df1[feature_to_plot].value_counts()

# Filter out categories with a count lower than X
X = 200
value_counts = value_counts[value_counts >= X]

# Create the bar plot
plt.barh(value_counts.index, value_counts)
value_counts.index
# Add axis labels and a title
plt.xlabel('Count')
plt.ylabel(feature_to_plot)
plt.title('{} Distribution (count >= {})'.format(feature_to_plot.capitalize(), X))

# Show the plot
plt.show()


### EDA

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming your DataFrame is named "df"
features = ['li_tags_sum','rating_count','rating_link','car_brand','year', 'mpg', 'Engine_size', 'price', 'Exterior_color', 'Interior_color', 'Drivetrain', 'Fuel_type', 'Transmission', 'Mileage']

# Select the three columns to show correlation
cols = ['li_tags_sum','rating_count','rating_link','car_brand','year', 'mpg', 'Engine_size', 'price', 'Exterior_color', 'Interior_color', 'Drivetrain', 'Fuel_type', 'Transmission', 'Mileage']

corr = df[cols].corr()

# Plot a heatmap of the correlation matrix
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.savefig('heatmap.jpg', dpi=300)
plt.show()


In [ ]:
import pandas as pd
import seaborn as sns


features = ['li_tags_sum','rating_count','rating_link','car_brand','year', 'mpg', 'Engine_size', 'price', 'Exterior_color', 'Interior_color', 'Drivetrain', 'Fuel_type', 'Transmission', 'Mileage']

# Select only the numeric columns for the scatter matrix plot
numeric_cols = ['li_tags_sum','rating_count','rating_link','car_brand','year', 'mpg', 'Engine_size', 'price', 'Exterior_color', 'Interior_color', 'Drivetrain', 'Fuel_type', 'Transmission', 'Mileage']
numeric_df = df[numeric_cols]

# Create the scatter matrix plot using seaborn
sns.pairplot(numeric_df)

# Save the plot as a jpeg file
plt.savefig('scatter_matrix.jpg', dpi=300)



In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming your DataFrame is named "df"
features = ['car_brand','year', 'mpg', 'Engine_size', 'price', 'Exterior_color', 'Interior_color', 'Drivetrain', 'Fuel_type', 'Transmission', 'Mileage']

# Select only the numeric columns for the scatter matrix plot
numeric_cols = ['rating_count','year', 'mpg', 'Engine_size', 'price', 'Mileage','car_brand','Drivetrain','Fuel_type']
numeric_df = df[numeric_cols]

# Iterate through the columns of numeric_df and create a separate scatter plot for each column
for col in numeric_df.columns:
    sns.scatterplot(x='rating_count', y=col, data=numeric_df)
    plt.xticks(rotation=90)  # Rotate the x-axis labels by 90 degrees
    plt.show()


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming your DataFrame is named "df"
features = ['li_tags_sum','rating_count','rating_link','car_brand','year', 'mpg', 'Engine_size', 'price', 'Exterior_color', 'Interior_color', 'Drivetrain', 'Fuel_type', 'Transmission', 'Mileage']

# Select the three columns to show correlation
cols = ['li_tags_sum','rating_count','rating_link','car_brand','year', 'mpg', 'Engine_size', 'price', 'Exterior_color', 'Interior_color', 'Drivetrain', 'Fuel_type', 'Transmission', 'Mileage']

# Compute correlation matrix
corr = df[cols].corr()

# Heatmap of correlation matrix
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.title('Heatmap of correlation matrix')
plt.savefig('heatmap.jpg', dpi=300)
plt.show()

# Scatterplot matrix
sns.pairplot(df[cols])
plt.title('Scatterplot matrix')
plt.savefig('scatterplot_matrix.jpg', dpi=300)
plt.show()

# Pairplot
sns.pairplot(df[cols], diag_kind='hist')
plt.title('Pairplot')
plt.savefig('pairplot.jpg', dpi=300)
plt.show()

# Correlation matrix plot
sns.clustermap(corr, cmap='coolwarm', annot=True)
plt.title('Correlation matrix plot')
plt.savefig('correlation_matrix.jpg', dpi=300)
plt.show()

# Regression plot
sns.regplot(x='price', y='Engine_size', data=df)
plt.title('Regression plot')
plt.savefig('regression_plot.jpg', dpi=300)
plt.show()

# Jointplot
sns.jointplot(x='price', y='Engine_size', data=df, kind='hex')
plt.title('Jointplot')
plt.savefig('jointplot.jpg', dpi=300)
plt.show()


### lets find the best TREE for us!

In [ ]:
import itertools
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

def process_combinations(features, df, max_depth=None):
    max_r2 = 0
    for i in range(len(features)):
        combinations = itertools.combinations(features, i+1)
        for combo in combinations:
            df1 = df.copy()
            r2 = process_combination(combo, df1, max_depth=max_depth)
            if r2 > max_r2:
                max_r2 = r2
    print(f'Coefficient of determination (R^2)_MAX_IS : {max_r2:.2f}')

def process_combination(combo, df, max_depth=None):
    # select the features to use as input variables
    features = list(combo)
    
    # drop rows with missing values
    df = df.dropna()
    
    # split the data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(df[features], df['rating_count'], test_size=0.5, random_state=42)
    
    # create and fit the decision tree model
    model = DecisionTreeRegressor(max_depth=max_depth)
    model.fit(X_train, y_train)
    
    # make predictions on the test set
    y_pred = model.predict(X_test)
    
    # evaluate the model
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    print(f'Features: {features}')
    print(f'Mean squared error: {mse:.2f}')
    print(f'Coefficient of determination (R^2): {r2:.2f}')
    
    return r2


In [ ]:
features = ['car_brand','year', 'mpg','Engine_size','rating_link','price','Exterior_color','Interior_color','Drivetrain','Fuel_type','Transmission','Mileage']
df = df.dropna()
process_combinations(features,df)


### changing 
all number of reviews

 Features: ['car_brand', 'year', 'rating_link', 'price', 'Transmission', 'Mileage']
 Mean squared error: 0.01
 Coefficient of determination (R^2): 0.31

### changing 
 number of reviews >10 

Features: ['year', 'Engine_size', 'rating_link', 'Mileage']
Mean squared error: 0.00
Coefficient of determination (R^2): 0.36

In [ ]:
df